In [215]:
import pandas as pd
import numpy as np
import dtale
import dtale.app as dtale_app
import requests
from pandas_datareader.data import DataReader
from datetime import datetime
#set pd for 2 decimal float 
pd.set_option('display.float_format', lambda x: '%.2f' % x)
#functiion for dtale 
kill=!sudo kill -9 $(sudo lsof -t -i:40000)
print('No active dtale')

instance_lst =[]
def create_dtale (df,name):
    if not (name in instance_lst) :
        dtale.show(df,name=name)
        instance_lst.append(name)
        dtale.instances()
    else :
        print('Company already exists!')
        dtale.instances() 

def get_data(file):
    df = pd.read_csv('StrategyReports_AAPL_652122.csv',sep=';',skiprows=4)
    df['Date/Time']=pd.to_datetime(df['Date/Time'],format='%m/%d/%y, %I:%M %p')
    df_data=df.Strategy.str.split("|",expand=True).drop(labels=[0,5], axis=1)
    df_data.columns=['Open','High','Low','Close']
    df.drop(labels=['Id','Amount','Strategy','Side','Price','Trade P/L','P/L','Position','Unnamed: 9'],inplace=True, axis=1)
    df=pd.concat([df_data, df], axis=1)
    df=df.set_index(['Date/Time']).iloc[1:-2:2]
    df=df.rename(columns={'Date/Time':'Date'})
    return df

No active dtale


In [216]:
symbol=input('Please Enter Stock Symbol : ').upper()
file=input('Please Enter file Name : ')
df=get_data(file)
df.to_csv(symbol+'.csv',index=True)
df=pd.read_csv(symbol+'.csv')
df.tail(1)
df.rename(columns={'Date/Time':'Date'})

Please Enter Stock Symbol :  AAPL
Please Enter file Name :  StrategyReports_AAPL_652122


,Date,Open,High,Low,Close
0,2021-05-28 09:31:00,125.56,125.80,125.30,125.30
1,2021-05-28 09:32:00,125.32,125.32,124.95,125.02
2,2021-05-28 09:33:00,125.03,125.17,124.95,125.00
3,2021-05-28 09:34:00,125.00,125.05,124.74,124.77
4,2021-05-28 09:35:00,124.77,124.88,124.58,124.83
...,...,...,...,...,...
1944,2021-06-04 15:55:00,125.92,125.98,125.92,125.94
1945,2021-06-04 15:56:00,125.94,125.95,125.85,125.86
1946,2021-06-04 15:57:00,125.86,125.91,125.85,125.86
1947,2021-06-04 15:58:00,125.86,125.87,125.81,125.86


In [218]:
#Add calulated Columns 
# add calculated col Candle stick 
df['Candle'] = df.Close - df.Open
df['Ch%']=(df.Candle/df.Open)*100
#Highs percent of open price 
df['H%']=((df.High-df.Open)/df.Open)*100

#lows percent of open price
df['L%']=((df.Open-df.Low)/df.Open)*100

#High - low value 
df['HL']=df.High-df.Low
df['HLCH%']=(df.HL/df.Open)*100

#Low to close 
df['LtC']=df.Close-df.Low
df['LtC%']=(df.LtC/df.Open)*100

'''# Converte date col to date type then get day name by date value 
df['Date'] = pd.to_datetime(df['Date'])
df['Day'] = df['Date'].dt.day_name()'''
#Calculate mean , median 

# apply up trend if Candle is postive other is negative 
df['UPTrend'] = np.where(df['Candle']>0, True, False)

#Filter unwanted column
FilteredDf= df#[['Date','Open','Candle','Ch%','H%','L%','HL','HLCH%','LtC%','Day','UPTrend']]
uptrend= FilteredDf.loc[df['UPTrend'] == True]
downtrend= FilteredDf.loc[df['UPTrend'] == False]

create_dtale(df=df,name=symbol)


To gain access to an instance object simply pass the value from 'ID' to dtale.get_instance(ID)

  ID Name                                       URL
   1 AAPL http://ip-172-31-8-226:40000/dtale/main/1
aapl AAPL http://ip-172-31-8-226:40000/dtale/main/1
Executing shutdown due to inactivity...


2021-06-05 17:35:05,161 - INFO     - Executing shutdown due to inactivity...


Executing shutdown...


2021-06-05 17:35:05,212 - INFO     - Executing shutdown...
